# PROJECT

project description

References used

Visual drill
https://cloud.google.com/looker/docs/best-practices/how-to-use-more-powerful-data-drilling

instructions to code visual drilling
https://www.googlecloudcommunity.com/gc/Exploring-Curating-Data/How-to-get-visualization-configurations-for-custom-drilling/td-p/592014

Looker python SDK tutorial (Instructions and code cells taken from this notebook)
https://colab.research.google.com/github/looker-open-source/sdk-codegen/blob/main/python/python-sdk-tutorial.ipynb#scrollTo=vEbO5uuFQnSn



# Installing Looker SDK
We'll install the latest looker_sdk from [pypi](https://pypi.org/project/looker-sdk/).



In [ ]:
%%capture
!pip install looker_sdk

In [ ]:
import looker_sdk
import os # We import os here in order to manage environment variables.
import json
import vertexai
from vertexai.preview.generative_models import GenerativeModel, GenerationConfig
import configparser
import urllib.parse
import re

# Configuring & Initializing the SDK
Environment variables to configure the SDK.



In [ ]:
from google.colab import userdata


os.environ["LOOKERSDK_BASE_URL"] = "https://hack.looker.com:19999" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "4.0" #As of Looker v23.18+, the 3.0 and 3.1 versions of the API are removed. Use "4.0" here.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  userdata.get('LOOKERSDK_CLIENT_ID') # Provided by Looker hackathon org team
os.environ["LOOKERSDK_CLIENT_SECRET"] = userdata.get('LOOKERSDK_CLIENT_SECRET') # Provided by Looker hackathon org team

print("All environment variables set.")

All environment variables set.


In [ ]:
sdk = looker_sdk.init40()
print('Looker SDK 4.0 initialized successfully.')

Looker SDK 4.0 initialized successfully.


# APP: LookML metric with visual drill
It usually helps to mock up the flow of an App/script before diving into code. In this case, our mission is to take the output of one Look and pipe it into the filters of a second Look. That means it'll look something like this:

1. Enter Look ID

Let's take it item by item.

In [ ]:
#@title Look ID
#@markdown Enter a look ID (`look_id`) that you want to apply as visual drill.

#@markdown If your URL is https://your.cloud.looker.com/looks/25, your Look ID is 25.
#@markdown The Look can have pivots and filters.

look_id = "66" #@param {type:"string"}
title = "Hackathon Metric Visual Drill" #@param {type:"string"}

look = sdk.look(look_id=look_id)
query = look.query # Look query

print(f"Look ID selected: {look.id}.\n")

print ('Look Summary: \n')
try:
  response = sdk.run_look(
    look_id=look.id,
    result_format= "json" # Options here are csv, json, json_detail, txt, html, md, xlsx, sql (returns the raw query), png, jpg. JSON is the easiest to work with in python, so we return it.
  )
  data = json.loads(response) #The response is just a string, so we have to use the json library to load it as a json dict.
  print(data) #If our query was successful we should see an array of rows.
except:
  raise Exception(f'Error running look {look.id}')

Look ID selected: 66.

Look Summary: 

[{'flights.states': 'TX', 'flights.count': 482}, {'flights.states': 'OK', 'flights.count': 132}, {'flights.states': 'AR', 'flights.count': 76}, {'flights.states': 'LA', 'flights.count': 29}]


# APP Code

**1. Get params #1 from Look #1**

Example Look created in hack Looker instance:
https://hack.looker.com/looks/60

More complex example (with pivots and filters)
https://hack.looker.com/looks/61


In [ ]:
def vis_config_formatter(query_vis_config):
    # extracting the vis_params from json string
    json_string = json.dumps(query_vis_config)
    # formatting, adapt to lookML
    vis_params_from_json = json_string.replace(",", ",\n")
    vis_params_from_json = vis_params_from_json.replace('"', '\\"')
    return vis_params_from_json[1:-1]

In [ ]:
def filters_dict_to_link(filters):
    output_string = ""
    for key, value in filters.items():
      # Escape any dots in the key for f-string formatting
      output_string += f"f[{key}]={value}&"
    # Remove the trailing ampersand (&)
    output_string = output_string[:-1]
    return output_string

In [ ]:
def tail_builder(filters = {}, pivots = []):
    tail = '&toggle=dat,pik,vis&limit=5000'
    if not pivots:
        pivots = []
    if not filters:
        filters = {}
    pivots_string = ",".join(map(str, pivots))
    head_pivots = '&pivots=' + pivots_string
    head_filter = '&' + filters_dict_to_link(filters)
    if pivots and not filters:
        tail = head_pivots + tail
    if filters and not pivots:
        tail = head_filter + tail
    if filters and pivots:
        tail = head_pivots + head_filter + tail
    return tail

In [ ]:
def link_builder(label, vis_params, filters=None, pivots=None):

    tail = tail_builder(filters, pivots)

    template_link = f"""link: {{
    label: "{label}"
    url: "
    {{% assign vis_config = '{{
    {vis_params}
    }}' %}}
    {{{{ link }}}}&vis_config={{{{ vis_config | encode_uri }}}}{tail}"
    }}
    """
    link = template_link
    return link

In [ ]:
vis_params = vis_config_formatter(query.vis_config)
#print(vis_params)

In [ ]:
link = link_builder(title, vis_params, pivots=query.pivots, filters=query.filters)
print(link)

link: {
    label: "Hackathon Metric Visual Drill"
    url: "
    {% assign vis_config = '{
    \"value_labels\": \"legend\",
 \"label_type\": \"labPer\",
 \"x_axis_gridlines\": false,
 \"y_axis_gridlines\": true,
 \"show_view_names\": false,
 \"show_y_axis_labels\": true,
 \"show_y_axis_ticks\": true,
 \"y_axis_tick_density\": \"default\",
 \"y_axis_tick_density_custom\": 5,
 \"show_x_axis_label\": true,
 \"show_x_axis_ticks\": true,
 \"y_axis_scale_mode\": \"linear\",
 \"x_axis_reversed\": false,
 \"y_axis_reversed\": false,
 \"plot_size_by_field\": false,
 \"x_axis_zoom\": true,
 \"y_axis_zoom\": true,
 \"trellis\": \"\",
 \"stacking\": \"percent\",
 \"limit_displayed_rows\": false,
 \"legend_position\": \"center\",
 \"series_types\": {},
 \"point_style\": \"none\",
 \"series_labels\": {},
 \"show_value_labels\": false,
 \"label_density\": 25,
 \"x_axis_scale\": \"auto\",
 \"y_axis_combined\": true,
 \"ordering\": \"none\",
 \"show_null_labels\": false,
 \"show_totals_labels\": fals

In [ ]:
def metric_builder(fields):

    link = link_builder(title, vis_params, pivots=query.pivots, filters=query.filters)

    metric_template = f"""measure: metric_with_visual_drill {{
        type: count
        label: "{title}"
        drill_fields: [{", ".join(fields)}]
        {link}
      }}"""

    return metric_template.replace('.count','.metric_with_visual_drill')

# OUTPUT METRIC



In [ ]:
metric = metric_builder(query.fields)
print(metric)

measure: metric_with_visual_drill {
        type: count
        label: "Hackathon Metric Visual Drill"
        drill_fields: [flights.metric_with_visual_drill, flights.states]
        link: {
    label: "Hackathon Metric Visual Drill"
    url: "
    {% assign vis_config = '{
    \"value_labels\": \"legend\",
 \"label_type\": \"labPer\",
 \"x_axis_gridlines\": false,
 \"y_axis_gridlines\": true,
 \"show_view_names\": false,
 \"show_y_axis_labels\": true,
 \"show_y_axis_ticks\": true,
 \"y_axis_tick_density\": \"default\",
 \"y_axis_tick_density_custom\": 5,
 \"show_x_axis_label\": true,
 \"show_x_axis_ticks\": true,
 \"y_axis_scale_mode\": \"linear\",
 \"x_axis_reversed\": false,
 \"y_axis_reversed\": false,
 \"plot_size_by_field\": false,
 \"x_axis_zoom\": true,
 \"y_axis_zoom\": true,
 \"trellis\": \"\",
 \"stacking\": \"percent\",
 \"limit_displayed_rows\": false,
 \"legend_position\": \"center\",
 \"series_types\": {},
 \"point_style\": \"none\",
 \"series_labels\": {},
 \"show_valu